# Django Intro - Views & Http Responses

So, to pick up where we left off, we are going to delete the models module we created for practicing models and mirgrating them. We will then create a new module with the django command line interface(CLI) 

The app that we are going to create is called `pollingplace`. For now it will only have one simple view that will display text to the screen when the URL route is hit. 

SO, here's what we need to do first:

    - python manage.py startapp pollingplace #command prompt line
    - Add the following file
        pollingplace/
            urls.py
            
 Add these lines of code to `views.py`:
 
     from django.shortcuts import render
     from django.http import HttpResponse

        def index(request):
            return HttpResponse("Hello, world. It's Joel!!! You're at the polling place!!")
            
            
Inside of `pollingplace/urls.py`:

    # Custom imports

    from django.conf.urls import url
    from . import views


    # Create your views here.
    urlpatterns = [
        url(r'^$', views.index, name='index'),
        ]

Inside of `polls/urls.py` add the following line(s):

    #Django default imports
    from django.conf.urls import include,url
    from django.contrib import admin

    urlpatterns = [
        url(r'^polls/', include('pollingplace.urls')),
        url(r'^admin/', admin.site.urls),

    ]


#### Remember to add `pollingplace` to your INSTALLED_APPS in the `settings.py` file

Once you have everything in it's respective places, go ahead and run your server again with the `python manage.py runserver` command

BRILLIANT!! Our first view works!! SO, let's start creating a model for this app.

In [ ]:
# Head over to pollingplace/models.py
# We need to add the following lines of code for our models

from django.db import models

#Create your models underneath this import

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField(verbose_name='date published')
    
class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default = 0)

Cool, now we have a model that we can work with. 

One quick note here. Although we already specified pollingplace inside of INSTALLED APPS to test our HTML HTTP Response, a more specified way to tell the base app(polls) specific info about other apps, we should add the full dot seperated path to the `pollingplace` app file. 

To do this we simply replace `pollingplace` with `pollingplace.apps.PollingplaceConfig`. 

We could have gotten away with just using `pollingplace` but the best practice here is to use the full dot seperated path when available inside of INSTALLED_APPS.

##### Now lets create a migration for pollingplace.


In [ ]:
# Inside of the command prompt type the following command

python manage.py makemigrations pollingplace


'''
The above code should give an output like this:

Migrations for 'pollingplace':
  pollingplace\migrations\0001_initial.py
    - Create model Choice
    - Create model Question
    - Add field question to choice

'''

#### Let's checkout what we just created under the hood in SQL

In [ ]:
''' Running the following command will show you the full execution of what we just created 
and what will happen when we decide to migrate our current model '''

# python manage.py sqlmigrate pollingplace 0001

'''
The result should be:

BEGIN;
--
-- Create model Choice
--
CREATE TABLE "pollingplace_choice" ("id" serial NOT NULL PRIMARY KEY, "choice_text" varchar(200) NOT NULL, "votes" integer NOT NULL);
--
-- Create model Question
--
CREATE TABLE "pollingplace_question" ("id" serial NOT NULL PRIMARY KEY, "question_text" varchar(200) NOT NULL, "pub_date" timestamp with time zone NOT NULL);
--
-- Add field question to choice
--
ALTER TABLE "pollingplace_choice" ADD COLUMN "question_id" integer NOT NULL;
CREATE INDEX "pollingplace_choice_question_id_9dc0773f" ON "pollingplace_choice" ("question_id");
ALTER TABLE "pollingplace_choice" ADD CONSTRAINT "pollingplace_choice_question_id_9dc0773f_fk_pollingpl" FOREIGN KEY ("question_id") REFERENCES "pollingplace_question" ("id") DEFERRABLE INITIALLY DEFERRED;
COMMIT;
'''

So basically what we just did was create a migration which then perfoms a transaction for our database. Hopefully you remember transactions and how useful they are, but if not...the main takeaway here is that if something happens to go wrong while this transaction(migration) is happening, everything will stop and the database won't be effected.

Dope!! So now let's migrate this over and then start playing around with the admin backend of our app.

### BUT FIRST.....

We need to create a superuser for the admin panel. We can accomplish this using the following syntax:

    python manage.py createsuperuser
    
You should then be asked for to create a username, give an email, and create a password.

Once you have created this user account you will need to run the server again...this time navigating to `localhost:8000/admin`

Once there, go ahead and sign in with the newly created user account. 